# GPT architecture

## Transformer architecture


First off, we have the inputs and outputs (shifted right). goes througth the embeddings (POS encoding). All this passes through number of layers. lets take 4 layers, So, inputs will go through 4 encoders and then outputs and final encoder values passes through the 4 decoders. then the last decoder will do a transformation and pass through the softmax and then give probability sampling generation

Second off, in each encoder there going to be four process:

- Mulihead attention
  
  There are a bunch of heads learning different semantic info from a unique perspective and all the heads have different leanable parameters. Lets see the inner workings :

     - Keys, Query, Value
       
       Basically, key is like all the words in a sentence and query is like the word trying to find and when they dot product and gives a attention score.

     - Scaled Dot-product attention (n heads all run in parallel) 

       Lets see how the one head works, first the k and q is dot product and its attention score is formed and also these values are scaled so we dont get high values and then sometimes we do torch trill masking and then we do softmax for getng more confidence in the attention scoore and then we do matrix multiply with the v value which a output of input vector values and attention placed on each token.

     - Concatenate results
     - Linear

- Residual connection (add) then normalize

- Feed Forward (Linear -> ReLU -> Linear)

- Residual connection then normalize

Feeds into the next encoder block. if its the last encoder block it feed into each decoder block

multi-head atention - we look at past, present and future
masked multi-head attention - we look at past and present only


## Self-Attention working

So, lets take a four token sequence "My dog has fleas". We are going to hihlight which words are going to correlating eachother or attention mechanism is going multipy them together and get a high score. lets see how GPT understands from the inside.

	     My	 dog	 has	fleas  
My	   low	 med	 low	  low
dog	   med   low   med    high
has	   low   med   low    med
fleas	 low   high  med    low

So from these you can see high and low values, which makes the network learn how to place the attention score on the tokens and thses learn through embeddings. Attention is used to generate tokens, thats how the gpt works.

## GPT architecture

Basically gpt works same as the transformer architecture but the encoder part is just removed.

Also in attention we use scaling in the attention scores so we can protect the model from vanishing gradient problem doesnt attack.

Let me explian it onw more time:

GPT process :

- Tokenized outputs
- Embeddings + POS encoding
- n Decoders according to the layers
    In each decoder the process is :
       - Multihead attention
           n heads of multihead runs in parallel.
            the multihead process is :
          - Keys, Query and Value
          - Scaled dot product attention (head)
                   Basically, in each head the key and query will be done dot product and then its scaled to combat the vanishig gradient after that torch.trill masking for not lookin into the future and then softmax then the value and softmax will matrix multiply. the output will be a blend of inputs vector and attention placed on each token.
          - Conconate results
          - Linear
       - Residual connection (Add) then normalise
       - Feed Forward  (Linear -> ReLU -> Linear)
       - Residual connection (Add) and normalise
- nnLinear
- Softmax
- Probability sampling generation
- Compare targets to outputs -> backpropogate


## GPT Code

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import mmap
import random
import pickle

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

block_size = 64
batch_size = 128
max_iters = 3000
learning_rate = 3e-4
eval_iters = 100
eval_interval = 500
n_embd = 384
n_layer = 8
n_head = 8
dropout = 0.2


print(device)

cuda


In [3]:
chars = ''
with open('vocab.txt','r',encoding='utf-8') as f:
    text = f.read()
    chars = sorted(list(set(text)))

vocab_size = len(chars)

In [4]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s ]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [5]:
# memory map for using small snippets of text from a single file of any size
def get_random_chunk(split):
    filename = "train.txt" if split == 'train' else "val.txt"
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            # Determine the file size and a random position to start reading
            file_size = len(mm)
            start_pos = random.randint(0, (file_size) - block_size*batch_size)

            # Seek to the random position and read the block of text
            mm.seek(start_pos)
            block = mm.read(block_size*batch_size-1)

            # Decode the block to a string, ignoring any invalid byte sequences
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')
            
            # Train and test splits
            data = torch.tensor(encode(decoded_block), dtype=torch.long)
            
    return data


def get_batch(split):
    data = get_random_chunk(split)
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [6]:
@torch.no_grad   #this is used to make the computation easier by removing gradients

def estimate_loss():
    out = {}
    model.eval()  # to make the model enter the evaluation mode
    for split in ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits , loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

in the gpt model we have to do positional encoding to the tokenized inputs, but if we put the positional encoding its a fixed function. which does not learn about the data. so we going to put the positional encoding inside the nn.embedding.

self.blocks is the variable containing the sequential running of the decoders of 4 layer using the n_layers
LayerNOrm is used to end of the netowrk to make the model to converge better and normalize
linear it there so it can make the final outpu workable with the softmax

while doing in practice sinisudal embeddings are used to transformer models and while doing GPT they use learnable embeddings

im the forawrd pass, we are going to add a code where we are going to create token embedding and positional embedding and add them both and then run blocks on then do layernorm and then do the linear function

In [7]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.values = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5
        wei = wei.masked_fill(self.tril[:T,:T] == 0, float('-inf'))
        wei = F. softmax(wei, dim= -1)
        wei = self.dropout(wei)
        v = self.values(x)
        out = wei @ v
        return out


class MultiHeadAttention(nn.Module):
    def __init__(self, n_head, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(n_head)])
        self.proj = nn.Linear(head_size * n_head, n_embd)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout), #certain percentage of neurons randomly removed to prevent overfitting
        )
    
    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    def __init__(self,n_embd,n_head):
        super().__init__()
        head_size = n_embd // n_head         # head size si number of features each head will capture 
        self.sa = MultiHeadAttention(n_head,head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    
    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x+y)
        return x


class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,n_embd)  # we are creating a embedding table with dimentional of vocab_size and n_embed
        self.positional_embedding_table = nn.Embedding(block_size, n_embd)  #we are creating embedding table for positional encoding using the block size and n_embed
        self.blocks = nn.Sequential(*(Block(n_embd, n_head=n_head) for _ in range(n_layer))) 
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.apply(self._init_weights)
    
    def _init_weights(self,module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)



    def forward(self, index, targets=None):
       
        logits = self.token_embedding_table(index)
        B, T, C = logits.shape

        tok_emb = self.token_embedding_table(index)
        pos_emb = self.positional_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape  # B - Batch, T - Time, C - Channel (size of sequence)
            logits = logits.view(B*T,C) 
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)  # we call the forward pass
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)   #we get the probability distribution and dimension is -1 because we want the +1 index expected index
            index_next = torch.multinomial(probs, num_samples=1) #takes the highest number of prob
            index = torch.cat((index, index_next),dim=1) # concatinates to the next element the whole size
        return index
    
model = GPTLanguageModel(vocab_size)
m = model.to(device)

In [8]:
import torch

# Assuming the necessary components like model, estimate_loss, get_batch, learning_rate, max_iters, and eval_iters are defined

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# List to store all evaluations
all_evaluations = []

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        evaluation_str = f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}"
        all_evaluations.append(evaluation_str)
        print("\n\n")
        print(evaluation_str)
        print("\n\n")

    xb, yb = get_batch('train')
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

# Print all evaluations
print("\n\nAll Evaluations:\n")
for eval_str in all_evaluations:
    print(eval_str)
    print("\n")

with open('/home/ntejha/Desktop/Master/TaxEvade/model-01.pkl','wb') as f:
    pickle.dump(model,f)





step: 0, train loss: 10.398, val loss: 10.397






step: 100, train loss: 2.402, val loss: 2.437






step: 200, train loss: 2.272, val loss: 2.254






step: 300, train loss: 2.218, val loss: 2.131






step: 400, train loss: 2.003, val loss: 1.951






step: 500, train loss: 1.878, val loss: 1.896






step: 600, train loss: 1.848, val loss: 1.837






step: 700, train loss: 1.800, val loss: 1.733






step: 800, train loss: 1.742, val loss: 1.761






step: 900, train loss: 1.635, val loss: 1.766






step: 1000, train loss: 1.710, val loss: 1.698






step: 1100, train loss: 1.662, val loss: 1.687






step: 1200, train loss: 1.574, val loss: 1.626






step: 1300, train loss: 1.613, val loss: 1.580






step: 1400, train loss: 1.610, val loss: 1.593






step: 1500, train loss: 1.574, val loss: 1.587






step: 1600, train loss: 1.554, val loss: 1.523






step: 1700, train loss: 1.554, val loss: 1.518






step: 1800, train loss: 1.520, val loss: 1.533






